<h1>Finding a location for a german/bavarian-style restaurant in Munich</h1>

<p>Burkhardt Röper</p>

<h3>Install Packages</h3>

In [1]:
!pip install geopy # get geocoordinates
!pip install ratelimit # for time interval while getting geocoordinates
!pip install tqdm # for progress bar
!pip install ipyleaflet # map visualization

<h3>Crawl Munich District Data</h3>
<p>Neighborhood data comes from citypopulation.de, but has to be crawled. BeutifulSoup is used here.</p>

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# The crawling itself ...
source = requests.get('http://www.citypopulation.de/php/germany-munchen_d.php').text
soup = BeautifulSoup(source, 'lxml')

In [4]:
# Now let's iterate through the result, especially the table, who contains the data we are looking for
table=soup.find('table')

# Create the dataframe with pandas
column_names = ['Name','Status','2000', '2005', '2010', '2016']
df_stadtbezirksteil = pd.DataFrame(columns = column_names)

# Loop through table , get data, put it into dataframe 
for tr in table.find_all('tr'):
        
    rows=[]  
    
    for td in tr.find_all('td'):
        
        rows.append(td.text.strip())
    
    rows = rows[:-1]
    
    if len(rows)==6:
        df_stadtbezirksteil.loc[len(df_stadtbezirksteil)] = rows
        
        
# Let's have a look at the result
df_stadtbezirksteil.head(10)

,Name,Status,2000,2005,2010,2016
0,Allach - Untermenzing,Stadtbezirk,26.682,27.586,29.161,32.248
1,Industriebezirk,Stadtbezirksteil,12.922,12.811,13.810,15.264
2,Untermenzing-Allach,Stadtbezirksteil,13.760,14.775,15.351,16.984
3,Altstadt - Lehel,Stadtbezirk,18.374,18.631,19.207,21.454
4,Angerviertel,Stadtbezirksteil,3.204,3.243,3.261,3.663
5,Englischer Garten Süd,Stadtbezirksteil,4.187,4.372,4.544,5.022
6,Graggenau,Stadtbezirksteil,2.476,2.365,2.452,2.661
7,Hackenviertel,Stadtbezirksteil,1.681,1.650,1.605,2.029
8,Kreuzviertel,Stadtbezirksteil,541,582,559,756
9,Lehel,Stadtbezirksteil,6.285,6.419,6.786,7.323


<h3>Clean, correct, prepare data</h3>
<p>A lot to do here, as the data from citypopulation.de isn't perfect.</p>

In [5]:
df_stadtbezirksteil.drop(['2000', '2005', '2010'], axis=1, inplace=True) # drop years of population data we are not interested in

In [6]:
df_stadtbezirksteil.head()

,Name,Status,2016
0,Allach - Untermenzing,Stadtbezirk,32.248
1,Industriebezirk,Stadtbezirksteil,15.264
2,Untermenzing-Allach,Stadtbezirksteil,16.984
3,Altstadt - Lehel,Stadtbezirk,21.454
4,Angerviertel,Stadtbezirksteil,3.663


In [7]:
df_stadtbezirksteil.rename(columns={'2016': 'Population', 'Name': 'District'}, inplace=True) # renaming columns to easier understand the content

In [8]:
df_stadtbezirksteil.head()

,District,Status,Population
0,Allach - Untermenzing,Stadtbezirk,32.248
1,Industriebezirk,Stadtbezirksteil,15.264
2,Untermenzing-Allach,Stadtbezirksteil,16.984
3,Altstadt - Lehel,Stadtbezirk,21.454
4,Angerviertel,Stadtbezirksteil,3.663


In [9]:
# In the table there are also rows intended as headers for the boroughs.
# We don't need them, so I am putting them in a separate dataframe

condition_stadtbezirk = df_stadtbezirksteil['Status'] == 'Stadtbezirk'
df_stadtbezirk = df_stadtbezirksteil[condition_stadtbezirk]
df_stadtbezirk.drop(['Status'], axis=1, inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [10]:
# Now delete boroughs from our neighborhood-df
condition_stadtbezirk = df_stadtbezirksteil['Status'] != 'Stadtbezirk'
df_stadtbezirksteil = df_stadtbezirksteil[condition_stadtbezirk]

In [11]:
df_stadtbezirksteil.head(10)

,District,Status,Population
1,Industriebezirk,Stadtbezirksteil,15.264
2,Untermenzing-Allach,Stadtbezirksteil,16.984
4,Angerviertel,Stadtbezirksteil,3.663
5,Englischer Garten Süd,Stadtbezirksteil,5.022
6,Graggenau,Stadtbezirksteil,2.661
7,Hackenviertel,Stadtbezirksteil,2.029
8,Kreuzviertel,Stadtbezirksteil,756
9,Lehel,Stadtbezirksteil,7.323
11,Altaubing,Stadtbezirksteil,10.262
12,Aubing-Süd,Stadtbezirksteil,28.028


In [12]:
# We don't need the Status-column, so dropping
df_stadtbezirksteil.drop(['Status'], axis=1, inplace=True)

In [13]:
# The last row is irrelevant, so deleting
df_stadtbezirksteil = df_stadtbezirksteil[:-1]

<h3>Some manual correction, as the source contains some wrong and missing data</h3>
<p>One neighborhood was removed for being irrelevant. Unfortunately the neighborhoods Thalkirchen, Obersendling, Forstenried, Fürstenried and Solln were grouped into one borough at citypopulation.de. This grouping had to be undone manually.</p>

In [14]:
df_stadtbezirksteil = df_stadtbezirksteil[df_stadtbezirksteil['District'] != 'Land in Sonne']
df_stadtbezirksteil.at[98, 'District'] = 'Münchner Freiheit'

In [15]:
df_missing = pd.DataFrame({'District': ['Solln', 'Fürstenried', 'Forstenried', 'Obersendling', 'Thalkirchen'], 'Population': [20000, 19000, 19000, 19000, 19000]})
df_stadtbezirksteil = df_stadtbezirksteil.append(df_missing, ignore_index=True)

<h3>Find neighborhoods geocoordinates</h3>
<p>With the help of OpenStreetMap and associate tools</p>

In [16]:
from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="axelschweiss")

In [17]:
from ratelimit import limits, sleep_and_retry
from tqdm import tqdm
tqdm.pandas()

# The server only allows one request per second on the free tier, so this is the trick
@sleep_and_retry
@limits(1,1)
def rate_limited_geocode(query):
    return gc.geocode(query)

def geocode(row):
    # München and the name of the neighborhood works fine to retrieve geodata :-)
    lookup_query = 'München, ' + row["District"]
    
    lookup_result = rate_limited_geocode(lookup_query)
    
    return lookup_result

df_stadtbezirksteil['Geo_Coords'] = df_stadtbezirksteil.progress_apply(geocode, axis=1)
df_stadtbezirksteil.tail(10)

100%|██████████| 107/107 [01:45<00:00,  1.00s/it]


,District,Population,Geo_Coords
97,Giesing,14.303,"(Giesing, Stadtbezirk 18 Untergiesing-Harlachi..."
98,Harlaching,14.181,"(Harlaching, Stadtbezirk 18 Untergiesing-Harla..."
99,Neuharlaching,6.254,"(Neuharlaching, Stadtbezirk 18 Untergiesing-Ha..."
100,Siebenbrunn,6.563,"(Siebenbrunn, Bezirksteil Siebenbrunn, Stadtbe..."
101,Untergiesing,12.749,"(Untergiesing, Stadtbezirk 18 Untergiesing-Har..."
102,Solln,20000,"(Solln, Stadtbezirk 19 Thalkirchen-Obersendlin..."
103,Fürstenried,19000,"(Fürstenried, Stadtbezirk 19 Thalkirchen-Obers..."
104,Forstenried,19000,"(Forstenried, Stadtbezirk 19 Thalkirchen-Obers..."
105,Obersendling,19000,"(Obersendling, Stadtbezirk 19 Thalkirchen-Ober..."
106,Thalkirchen,19000,"(Thalkirchen, Stadtbezirk 19 Thalkirchen-Obers..."


In [21]:
# Creating own columns of lat/lng-Data
df_stadtbezirksteil['latlng'] = pd.DataFrame(df_stadtbezirksteil['Geo_Coords'].tolist())[1]
df_stadtbezirksteil = df_stadtbezirksteil['latlng'].apply(pd.Series, index=['lat', 'lng']).merge(df_stadtbezirksteil, left_index = True, right_index = True)

In [23]:
df_stadtbezirksteil.head()

,lat,lng,District,Population,Geo_Coords,latlng
0,48.196839,11.476602,Industriebezirk,15.264,"(Industriebezirk, Bezirksteil Industriebezirk,...","(48.1968392, 11.4766021)"
1,48.195994,11.457013,Untermenzing-Allach,16.984,"(Allach, Stadtbezirk 23 Allach-Untermenzing, M...","(48.1959936, 11.4570128)"
2,48.133670,11.571569,Angerviertel,3.663,"(Angerviertel, Bezirksteil Angerviertel, Stadt...","(48.1336701, 11.571569)"
3,48.147871,11.579682,Englischer Garten Süd,5.022,"(Bayerischer Verwaltungsgerichtshof, 23, Ludwi...","(48.1478711, 11.579682)"
4,48.139563,11.580182,Graggenau,2.661,"(Bezirksteil Graggenau, Stadtbezirk 01 Altstad...","(48.13956325, 11.5801820340781)"


In [24]:
# Looking for missing data ...
missing_data = df_stadtbezirksteil[~df_stadtbezirksteil.isnull()]
# ... not found :-)

,lat,lng,District,Population,Geo_Coords,latlng
0,48.196839,11.476602,Industriebezirk,15.264,"(Industriebezirk, Bezirksteil Industriebezirk,...","(48.1968392, 11.4766021)"
1,48.195994,11.457013,Untermenzing-Allach,16.984,"(Allach, Stadtbezirk 23 Allach-Untermenzing, M...","(48.1959936, 11.4570128)"
2,48.133670,11.571569,Angerviertel,3.663,"(Angerviertel, Bezirksteil Angerviertel, Stadt...","(48.1336701, 11.571569)"
3,48.147871,11.579682,Englischer Garten Süd,5.022,"(Bayerischer Verwaltungsgerichtshof, 23, Ludwi...","(48.1478711, 11.579682)"
4,48.139563,11.580182,Graggenau,2.661,"(Bezirksteil Graggenau, Stadtbezirk 01 Altstad...","(48.13956325, 11.5801820340781)"
5,48.135731,11.569955,Hackenviertel,2.029,"(Hackenviertel, Stadtbezirk 01 Altstadt-Lehel,...","(48.1357308, 11.5699553)"
6,48.139698,11.573209,Kreuzviertel,756,"(Kreuzviertel, Bezirksteil Kreuzviertel, Stadt...","(48.1396984, 11.573209)"
7,48.138712,11.588795,Lehel,7.323,"(Lehel, Stadtbezirk 01 Altstadt-Lehel, München...","(48.1387124, 11.5887948)"
8,48.165736,11.401493,Altaubing,10.262,"(Bezirksteil Altaubing, Stadtbezirk 22 Aubing-...","(48.16573575, 11.401493367987)"
9,48.143059,11.425644,Aubing-Süd,28.028,"(Bezirksteil Aubing-Süd, Stadtbezirk 22 Aubing...","(48.14305935, 11.425644159385)"


<h3>Visualize districts on map (with leaflet-package)</h3>
<p>I know leaflet from my web-coding-projects (Angular, Ionic, etc.). So glad there is a "port" in Python. It's perfect!</p>

In [25]:
import ipyleaflet
from ipyleaflet import Map, Marker, MarkerCluster
 
m = Map(center=(48.138449, 11.566554),zoom=5, scroll_wheel_zoom=True)

def create_marker(row):    
    
    name = row['District']
    lat_lon = (row['lat'], row['lng'])
    
    return Marker(location=lat_lon,
                    draggable=False,
                    title=name)

markers = df_stadtbezirksteil.apply(create_marker, axis=1)
marker_cluster = MarkerCluster(markers=tuple(markers.values))
m.add_layer(marker_cluster)

display(m)

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

<h3>Define Foursquare Credentials and Version</h3>
<p>With all the data ready, let's look at Foursquare for food places in Munich</p>

In [50]:
from pandas.io.json import json_normalize
import numpy as np

In [163]:
CLIENT_ID = '3CXVFG0QQXGT0TZPZ2RG2TKFXO4G4BF1NQXWZRIXHQBHODYR' # your Foursquare ID
CLIENT_SECRET = 'A2QI5QKSLZM30WC0YHBYJGMRV2HVSYLKF2DHJJ4E2N0PHV30' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
radius = 1000
section = 'food'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3CXVFG0QQXGT0TZPZ2RG2TKFXO4G4BF1NQXWZRIXHQBHODYR
CLIENT_SECRET:A2QI5QKSLZM30WC0YHBYJGMRV2HVSYLKF2DHJJ4E2N0PHV30


In [164]:
# An AIO-Function for getting the data from Foursquare. Took me some time to get through all the code

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        #create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            section)
            
        #make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],   
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [165]:
# Just to be save, working with a copy from here on
df_temp = df_stadtbezirksteil

,lat,lng,District,Population,Geo_Coords,latlng
0,48.196839,11.476602,Industriebezirk,15.264,"(Industriebezirk, Bezirksteil Industriebezirk,...","(48.1968392, 11.4766021)"
1,48.195994,11.457013,Untermenzing-Allach,16.984,"(Allach, Stadtbezirk 23 Allach-Untermenzing, M...","(48.1959936, 11.4570128)"
2,48.133670,11.571569,Angerviertel,3.663,"(Angerviertel, Bezirksteil Angerviertel, Stadt...","(48.1336701, 11.571569)"
3,48.147871,11.579682,Englischer Garten Süd,5.022,"(Bayerischer Verwaltungsgerichtshof, 23, Ludwi...","(48.1478711, 11.579682)"
4,48.139563,11.580182,Graggenau,2.661,"(Bezirksteil Graggenau, Stadtbezirk 01 Altstad...","(48.13956325, 11.5801820340781)"
5,48.135731,11.569955,Hackenviertel,2.029,"(Hackenviertel, Stadtbezirk 01 Altstadt-Lehel,...","(48.1357308, 11.5699553)"
6,48.139698,11.573209,Kreuzviertel,756,"(Kreuzviertel, Bezirksteil Kreuzviertel, Stadt...","(48.1396984, 11.573209)"
7,48.138712,11.588795,Lehel,7.323,"(Lehel, Stadtbezirk 01 Altstadt-Lehel, München...","(48.1387124, 11.5887948)"
8,48.165736,11.401493,Altaubing,10.262,"(Bezirksteil Altaubing, Stadtbezirk 22 Aubing-...","(48.16573575, 11.401493367987)"
9,48.143059,11.425644,Aubing-Süd,28.028,"(Bezirksteil Aubing-Süd, Stadtbezirk 22 Aubing...","(48.14305935, 11.425644159385)"


In [166]:
# Here the AIO-Function is actually called
munich_venues = getNearbyVenues(names=df_temp['District'],
                                   latitudes=df_temp['lat'],
                                   longitudes=df_temp['lng']
                                  )

In [167]:
# So many food places in Munich :-)
print(munich_venues.shape)
munich_venues.head()

(2407, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Untermenzing-Allach,48.195994,11.457013,Bäckerei Schuhmair,48.197175,11.459016,Bakery
1,Untermenzing-Allach,48.195994,11.457013,Zur Allacher Mühle,48.198411,11.457869,Bavarian Restaurant
2,Untermenzing-Allach,48.195994,11.457013,Gasthaus zur Schießstätte,48.195601,11.456130,Bavarian Restaurant
3,Untermenzing-Allach,48.195994,11.457013,Sicilia,48.193331,11.459387,Italian Restaurant
4,Untermenzing-Allach,48.195994,11.457013,Antica Osteria,48.198897,11.458101,Trattoria/Osteria


In [168]:
# Let's group them by neighborhoods
munich_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Alt Moosach,12,12,12,12,12,12
Altbogenhausen,8,8,8,8,8,8
Alte Heide - Hirschau,4,4,4,4,4,4
Alte Kaserne,13,13,13,13,13,13
Altperlach,3,3,3,3,3,3
Am Hart,5,5,5,5,5,5
Am Luitpoldpark,7,7,7,7,7,7
Am Riesenfeld,9,9,9,9,9,9
Am Schlachthof,58,58,58,58,58,58


In [169]:
# How many unique categories are there?
print('There are {} uniques categories.'.format(len(munich_venues['Venue Category'].unique())))

There are 93 uniques categories.


In [170]:
# Preparing the data for clustering

# one hot encoding
munich_onehot = pd.get_dummies(munich_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
munich_onehot['Neighborhood'] = munich_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [munich_onehot.columns[-1]] + list(munich_onehot.columns[:-1])
munich_onehot = munich_onehot[fixed_columns]

munich_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,Untermenzing-Allach,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Untermenzing-Allach,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Untermenzing-Allach,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Untermenzing-Allach,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Untermenzing-Allach,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [171]:
# Size still okay? Yes ...
munich_onehot.shape

(2407, 94)

In [172]:
# Very important for every analysis: Get the data standardized
munich_grouped = munich_onehot.groupby('Neighborhood').mean().reset_index()
munich_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,BBQ Joint,Bagel Shop,Bakery,...,Taco Place,Tapas Restaurant,Taverna,Thai Restaurant,Theme Restaurant,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Xinjiang Restaurant
0,Alt Moosach,0.000000,0.000000,0.000000,0.00,0.083333,0.000000,0.000000,0.000000,0.333333,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000
1,Altbogenhausen,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000
2,Alte Heide - Hirschau,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000
3,Alte Kaserne,0.000000,0.000000,0.000000,0.00,0.076923,0.000000,0.000000,0.000000,0.153846,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000
4,Altperlach,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Am Hart,0.000000,0.000000,0.000000,0.00,0.400000,0.000000,0.200000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Am Luitpoldpark,0.000000,0.000000,0.000000,0.00,0.000000,0.142857,0.000000,0.000000,0.142857,...,0.000000,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Am Riesenfeld,0.000000,0.000000,0.000000,0.00,0.111111,0.000000,0.000000,0.000000,0.222222,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Am Schlachthof,0.000000,0.034483,0.000000,0.00,0.068966,0.000000,0.000000,0.000000,0.103448,...,0.000000,0.000000,0.000000,0.017241,0.000000,0.000000,0.000000,0.000000,0.017241,0.000000
9,Am Waldfriedhof,0.000000,0.000000,0.000000,0.00,0.250000,0.000000,0.000000,0.000000,0.250000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [173]:
munich_grouped.shape

(103, 94)

In [174]:
# Now let's have a look at the top 5 food places per neighborhood
num_top_venues = 5

for hood in munich_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = munich_grouped[munich_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alt Moosach----
                venue  freq
0              Bakery  0.33
1         Pizza Place  0.08
2  Italian Restaurant  0.08
3    Asian Restaurant  0.08
4   Trattoria/Osteria  0.08


----Altbogenhausen----
                   venue  freq
0     Italian Restaurant  0.25
1      Indian Restaurant  0.12
2  Vietnamese Restaurant  0.12
3    Japanese Restaurant  0.12
4                 Bakery  0.12


----Alte Heide - Hirschau----
                       venue  freq
0                       Café  0.50
1         Turkish Restaurant  0.25
2                     Bakery  0.25
3          Afghan Restaurant  0.00
4  Middle Eastern Restaurant  0.00


----Alte Kaserne----
                 venue  freq
0               Bakery  0.15
1   Italian Restaurant  0.08
2                 Café  0.08
3  Fried Chicken Joint  0.08
4     Doner Restaurant  0.08


----Altperlach----
                      venue  freq
0         German Restaurant  0.67
1      Pakistani Restaurant  0.33
2         Afghan Restaurant  0.00
3  Me

               venue  freq
0  Trattoria/Osteria  0.25
1             Bakery  0.25
2  German Restaurant  0.25
3               Café  0.25
4  Afghan Restaurant  0.00


----Herzogpark----
                      venue  freq
0                      Food  0.33
1         Trattoria/Osteria  0.33
2                Restaurant  0.33
3  Mediterranean Restaurant  0.00
4       Palatine Restaurant  0.00


----Johanneskirchen----
               venue  freq
0         Restaurant  0.25
1          BBQ Joint  0.25
2   Greek Restaurant  0.25
3  German Restaurant  0.25
4  Afghan Restaurant  0.00


----Josephsburg----
                 venue  freq
0     Asian Restaurant  0.22
1               Bakery  0.22
2                 Café  0.11
3   Italian Restaurant  0.11
4  Bavarian Restaurant  0.11


----Josephsplatz----
                   venue  freq
0                 Bakery  0.15
1                   Café  0.15
2     Italian Restaurant  0.08
3  Vietnamese Restaurant  0.05
4      Trattoria/Osteria  0.05


----Kleinhesselohe

                       venue  freq
0                  Gastropub   0.5
1                       Café   0.5
2          Afghan Restaurant   0.0
3  Middle Eastern Restaurant   0.0
4                   Pet Café   0.0


----Solln----
                 venue  freq
0    German Restaurant   0.4
1    Indian Restaurant   0.2
2   Italian Restaurant   0.2
3        Deli / Bodega   0.2
4  Japanese Restaurant   0.0


----St. Benno----
                   venue  freq
0     Italian Restaurant  0.21
1       Asian Restaurant  0.11
2      German Restaurant  0.11
3  Vietnamese Restaurant  0.07
4       Doner Restaurant  0.07


----St. Paul----
                      venue  freq
0          Asian Restaurant  0.14
1                    Bakery  0.08
2        Italian Restaurant  0.08
3  Mediterranean Restaurant  0.05
4                Restaurant  0.05


----St. Ulrich----
               venue  freq
0   Doner Restaurant  0.12
1         Restaurant  0.12
2             Bakery  0.12
3        Pizza Place  0.06
4  Trattoria/Os

In [175]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [176]:
# Next step: A table with the 10 most common food places ("venues")
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = munich_grouped['Neighborhood']

for ind in np.arange(munich_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(munich_grouped.iloc[ind, :], num_top_venues)


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alt Moosach,Bakery,Fast Food Restaurant,Burrito Place,German Restaurant,Trattoria/Osteria,Asian Restaurant,Pizza Place,Italian Restaurant,Food,Dim Sum Restaurant
1,Altbogenhausen,Italian Restaurant,Vietnamese Restaurant,Indian Restaurant,Japanese Restaurant,Bakery,Restaurant,Café,Xinjiang Restaurant,English Restaurant,Dim Sum Restaurant
2,Alte Heide - Hirschau,Café,Turkish Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
3,Alte Kaserne,Bakery,Greek Restaurant,Fried Chicken Joint,Café,Middle Eastern Restaurant,Doner Restaurant,Italian Restaurant,Restaurant,Asian Restaurant,Indian Restaurant
4,Altperlach,German Restaurant,Pakistani Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [196]:
neighborhoods_venues_sorted

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,7,Alt Moosach,Bakery,Fast Food Restaurant,Burrito Place,German Restaurant,Trattoria/Osteria,Asian Restaurant,Pizza Place,Italian Restaurant,Food,Dim Sum Restaurant
1,2,Altbogenhausen,Italian Restaurant,Vietnamese Restaurant,Indian Restaurant,Japanese Restaurant,Bakery,Restaurant,Café,Xinjiang Restaurant,English Restaurant,Dim Sum Restaurant
2,4,Alte Heide - Hirschau,Café,Turkish Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
3,2,Alte Kaserne,Bakery,Greek Restaurant,Fried Chicken Joint,Café,Middle Eastern Restaurant,Doner Restaurant,Italian Restaurant,Restaurant,Asian Restaurant,Indian Restaurant
4,1,Altperlach,German Restaurant,Pakistani Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
5,0,Am Hart,Asian Restaurant,German Restaurant,BBQ Joint,Mexican Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
6,5,Am Luitpoldpark,Gastropub,Italian Restaurant,Austrian Restaurant,Taverna,Bakery,Café,Burger Joint,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant
7,7,Am Riesenfeld,Bakery,Fast Food Restaurant,Greek Restaurant,Asian Restaurant,Doner Restaurant,Restaurant,Cafeteria,Falafel Restaurant,English Restaurant,Dim Sum Restaurant
8,2,Am Schlachthof,Café,Bakery,Italian Restaurant,Doner Restaurant,Asian Restaurant,German Restaurant,Fast Food Restaurant,Burger Joint,Seafood Restaurant,African Restaurant
9,2,Am Waldfriedhof,Asian Restaurant,Italian Restaurant,Bakery,Café,Xinjiang Restaurant,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [177]:
# Import stuff for clustering and visualization with folium (just to use something different than leaflet)
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means for clustering
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

<h3>Cluster Districts</h3>

In [178]:
# Set number of clusters
kclusters = 10

munich_grouped_clustering = munich_grouped.drop('Neighborhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(munich_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 2, 4, 2, 1, 0, 5, 7, 2, 2], dtype=int32)

In [179]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Alt Moosach,Bakery,Fast Food Restaurant,Burrito Place,German Restaurant,Trattoria/Osteria,Asian Restaurant,Pizza Place,Italian Restaurant,Food,Dim Sum Restaurant
1,Altbogenhausen,Italian Restaurant,Vietnamese Restaurant,Indian Restaurant,Japanese Restaurant,Bakery,Restaurant,Café,Xinjiang Restaurant,English Restaurant,Dim Sum Restaurant
2,Alte Heide - Hirschau,Café,Turkish Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
3,Alte Kaserne,Bakery,Greek Restaurant,Fried Chicken Joint,Café,Middle Eastern Restaurant,Doner Restaurant,Italian Restaurant,Restaurant,Asian Restaurant,Indian Restaurant
4,Altperlach,German Restaurant,Pakistani Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
5,Am Hart,Asian Restaurant,German Restaurant,BBQ Joint,Mexican Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
6,Am Luitpoldpark,Gastropub,Italian Restaurant,Austrian Restaurant,Taverna,Bakery,Café,Burger Joint,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant
7,Am Riesenfeld,Bakery,Fast Food Restaurant,Greek Restaurant,Asian Restaurant,Doner Restaurant,Restaurant,Cafeteria,Falafel Restaurant,English Restaurant,Dim Sum Restaurant
8,Am Schlachthof,Café,Bakery,Italian Restaurant,Doner Restaurant,Asian Restaurant,German Restaurant,Fast Food Restaurant,Burger Joint,Seafood Restaurant,African Restaurant
9,Am Waldfriedhof,Asian Restaurant,Italian Restaurant,Bakery,Café,Xinjiang Restaurant,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [180]:
# Add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

munich_merged = df_temp

# Merge dfs to add latitude/longitude for each neighborhood
munich_merged = munich_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='District')

munich_merged.head(10) # check the last columns!


,lat,lng,District,Population,Geo_Coords,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,48.196839,11.476602,Industriebezirk,15.264,"(Industriebezirk, Bezirksteil Industriebezirk,...","(48.1968392, 11.4766021)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,48.195994,11.457013,Untermenzing-Allach,16.984,"(Allach, Stadtbezirk 23 Allach-Untermenzing, M...","(48.1959936, 11.4570128)",2.0,Bavarian Restaurant,Trattoria/Osteria,Italian Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
2,48.133670,11.571569,Angerviertel,3.663,"(Angerviertel, Bezirksteil Angerviertel, Stadt...","(48.1336701, 11.571569)",5.0,Café,German Restaurant,Afghan Restaurant,Asian Restaurant,Italian Restaurant,Burger Joint,Thai Restaurant,Pizza Place,Seafood Restaurant,Vietnamese Restaurant
3,48.147871,11.579682,Englischer Garten Süd,5.022,"(Bayerischer Verwaltungsgerichtshof, 23, Ludwi...","(48.1478711, 11.579682)",5.0,Café,Italian Restaurant,Bakery,Restaurant,Doner Restaurant,Breakfast Spot,Pizza Place,Chinese Restaurant,Thai Restaurant,Asian Restaurant
4,48.139563,11.580182,Graggenau,2.661,"(Bezirksteil Graggenau, Stadtbezirk 01 Altstad...","(48.13956325, 11.5801820340781)",5.0,Café,Bavarian Restaurant,German Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,French Restaurant,Pizza Place,Asian Restaurant,Burger Joint
5,48.135731,11.569955,Hackenviertel,2.029,"(Hackenviertel, Stadtbezirk 01 Altstadt-Lehel,...","(48.1357308, 11.5699553)",5.0,Café,Italian Restaurant,Bavarian Restaurant,German Restaurant,Pizza Place,Burger Joint,Thai Restaurant,Bakery,Restaurant,Afghan Restaurant
6,48.139698,11.573209,Kreuzviertel,756,"(Kreuzviertel, Bezirksteil Kreuzviertel, Stadt...","(48.1396984, 11.573209)",5.0,Café,Restaurant,German Restaurant,Bavarian Restaurant,Italian Restaurant,French Restaurant,Bakery,Deli / Bodega,Pizza Place,Salad Place
7,48.138712,11.588795,Lehel,7.323,"(Lehel, Stadtbezirk 01 Altstadt-Lehel, München...","(48.1387124, 11.5887948)",2.0,German Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Mediterranean Restaurant
8,48.165736,11.401493,Altaubing,10.262,"(Bezirksteil Altaubing, Stadtbezirk 22 Aubing-...","(48.16573575, 11.401493367987)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,48.143059,11.425644,Aubing-Süd,28.028,"(Bezirksteil Aubing-Süd, Stadtbezirk 22 Aubing...","(48.14305935, 11.425644159385)",2.0,Pide Place,Pizza Place,Italian Restaurant,Bakery,Sandwich Place,Xinjiang Restaurant,Eastern European Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant


<h3>Visualize clusters</h3>
<p>I could have used leaflet, but as Folium was the framework of this course, I am using Folium this time</p>

In [181]:
# Needed for the isnan-class
import math

In [182]:
# Create map
map_clusters = folium.Map(location=[48.138449, 11.566554], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(munich_merged['lat'], munich_merged['lng'], munich_merged['District'], munich_merged['Cluster Labels']):
    
    if math.isnan(cluster):
        continue
    else:        
        cluster = int(cluster)
        label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster-1],
            fill=True,
            fill_color=rainbow[cluster-1],
            fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<p>Checking all the clusters to find a property for everyone.</p>

In [183]:

munich_merged.loc[munich_merged['Cluster Labels'] == 0, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,11.649074,"(48.1692948, 11.649074)",0.0,Greek Restaurant,German Restaurant,BBQ Joint,Restaurant,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
28,11.533323,"(48.2159429, 11.533323)",0.0,Greek Restaurant,German Restaurant,Taverna,Bakery,Café,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner
34,11.486244,"(48.1197314, 11.4862435)",0.0,Greek Restaurant,German Restaurant,Asian Restaurant,Bakery,Café,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant
54,11.575836,"(48.1964066, 11.5758359)",0.0,Asian Restaurant,German Restaurant,BBQ Joint,Mexican Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
67,11.482380,"(48.1419974, 11.4823797)",0.0,Fast Food Restaurant,Indian Restaurant,Greek Restaurant,German Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
97,11.571791,"(48.1045459, 11.5717907)",0.0,Indian Restaurant,Greek Restaurant,Bakery,Café,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
104,11.494477,"(48.0848354, 11.4944766)",0.0,Greek Restaurant,Pizza Place,BBQ Joint,Ramen Restaurant,Bakery,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner


In [184]:
munich_merged.loc[munich_merged['Cluster Labels'] == 1, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,11.403227,"(48.1366708, 11.4032274)",1.0,German Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
26,11.626194,"(48.1719489, 11.6261939)",1.0,German Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
66,11.592113,"(48.09977105, 11.5921131656746)",1.0,German Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
71,11.625335,"(48.0957206, 11.6253349608721)",1.0,German Restaurant,Pakistani Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
75,11.668531,"(48.0834909, 11.6685311)",1.0,German Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [185]:
munich_merged.loc[munich_merged['Cluster Labels'] == 2, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,11.457013,"(48.1959936, 11.4570128)",2.0,Bavarian Restaurant,Trattoria/Osteria,Italian Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
7,11.588795,"(48.1387124, 11.5887948)",2.0,German Restaurant,Italian Restaurant,Japanese Restaurant,Mexican Restaurant,Sushi Restaurant,Restaurant,Café,Trattoria/Osteria,Seafood Restaurant,Mediterranean Restaurant
9,11.425644,"(48.14305935, 11.425644159385)",2.0,Pide Place,Pizza Place,Italian Restaurant,Bakery,Sandwich Place,Xinjiang Restaurant,Eastern European Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant
12,11.603245,"(48.1326788, 11.6032448)",2.0,Bakery,Italian Restaurant,German Restaurant,Café,Indian Restaurant,Mexican Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Spanish Restaurant,Doner Restaurant
16,11.612402,"(48.1378093, 11.6124016)",2.0,Italian Restaurant,Chinese Restaurant,Vietnamese Restaurant,Greek Restaurant,Mexican Restaurant,German Restaurant,Pizza Place,Café,Restaurant,Japanese Restaurant
17,11.583222,"(48.1255599, 11.5832222)",2.0,Bakery,German Restaurant,Italian Restaurant,Pizza Place,Fast Food Restaurant,Greek Restaurant,Turkish Restaurant,Gastropub,Asian Restaurant,Café
19,11.617461,"(48.12736565, 11.617461468779)",2.0,Italian Restaurant,Bakery,Indian Restaurant,Asian Restaurant,Doner Restaurant,Pizza Place,Middle Eastern Restaurant,Portuguese Restaurant,Café,Xinjiang Restaurant
21,11.605498,"(48.1457847, 11.6054976)",2.0,Italian Restaurant,Vietnamese Restaurant,Indian Restaurant,Japanese Restaurant,Bakery,Restaurant,Café,Xinjiang Restaurant,English Restaurant,Dim Sum Restaurant
24,11.614716,"(48.1594565, 11.6147163)",2.0,Trattoria/Osteria,Restaurant,Food,Xinjiang Restaurant,English Restaurant,Currywurst Joint,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
33,11.479158,"(48.1119949, 11.4791576)",2.0,German Restaurant,Bakery,Fast Food Restaurant,Asian Restaurant,Steakhouse,Sushi Restaurant,Sandwich Place,Trattoria/Osteria,Dim Sum Restaurant,English Restaurant


In [186]:
munich_merged.loc[munich_merged['Cluster Labels'] == 3, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,11.693763,"(48.1102388, 11.6937625)",3.0,BBQ Joint,Xinjiang Restaurant,Falafel Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


In [187]:
munich_merged.loc[munich_merged['Cluster Labels'] == 4, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
76,11.606023,"(48.1771046, 11.6060234)",4.0,Café,Turkish Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
93,11.657732,"(48.1131392, 11.6577319)",4.0,Bakery,Café,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
98,11.565078,"(48.0882932, 11.5650783)",4.0,Italian Restaurant,Café,Bakery,Xinjiang Restaurant,Falafel Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant


In [188]:
munich_merged.loc[munich_merged['Cluster Labels'] == 5, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,11.571569,"(48.1336701, 11.571569)",5.0,Café,German Restaurant,Afghan Restaurant,Asian Restaurant,Italian Restaurant,Burger Joint,Thai Restaurant,Pizza Place,Seafood Restaurant,Vietnamese Restaurant
3,11.579682,"(48.1478711, 11.579682)",5.0,Café,Italian Restaurant,Bakery,Restaurant,Doner Restaurant,Breakfast Spot,Pizza Place,Chinese Restaurant,Thai Restaurant,Asian Restaurant
4,11.580182,"(48.13956325, 11.5801820340781)",5.0,Café,Bavarian Restaurant,German Restaurant,Restaurant,Japanese Restaurant,Italian Restaurant,French Restaurant,Pizza Place,Asian Restaurant,Burger Joint
5,11.569955,"(48.1357308, 11.5699553)",5.0,Café,Italian Restaurant,Bavarian Restaurant,German Restaurant,Pizza Place,Burger Joint,Thai Restaurant,Bakery,Restaurant,Afghan Restaurant
6,11.573209,"(48.1396984, 11.573209)",5.0,Café,Restaurant,German Restaurant,Bavarian Restaurant,Italian Restaurant,French Restaurant,Bakery,Deli / Bodega,Pizza Place,Salad Place
13,11.596584,"(48.1305229, 11.5965839)",5.0,Café,Italian Restaurant,Bakery,German Restaurant,Indian Restaurant,French Restaurant,Turkish Restaurant,Thai Restaurant,Asian Restaurant,Bistro
14,11.595202,"(48.1361343, 11.5952016)",5.0,Café,French Restaurant,Italian Restaurant,Bakery,German Restaurant,Asian Restaurant,Sushi Restaurant,Burger Joint,Seafood Restaurant,Bavarian Restaurant
15,11.589286,"(48.1233516, 11.5892863)",5.0,Italian Restaurant,French Restaurant,Café,Restaurant,Vietnamese Restaurant,Turkish Restaurant,BBQ Joint,German Restaurant,Bakery,Bavarian Restaurant
27,11.591766,"(48.1806136, 11.5917664)",5.0,Restaurant,Italian Restaurant,Café,Cafeteria,Gastropub,Bistro,Bakery,Salad Place,BBQ Joint,Asian Restaurant
37,11.562986,"(48.1291891, 11.5629864)",5.0,Café,Asian Restaurant,Burger Joint,Vietnamese Restaurant,Italian Restaurant,Fast Food Restaurant,Bakery,Bavarian Restaurant,Restaurant,Xinjiang Restaurant


In [189]:
munich_merged.loc[munich_merged['Cluster Labels'] == 6, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,11.656920,"(48.1493975, 11.65692)",6.0,German Restaurant,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
29,11.550703,"(48.1960931, 11.5507034)",6.0,Trattoria/Osteria,German Restaurant,Café,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
30,11.550703,"(48.1960931, 11.5507034)",6.0,Trattoria/Osteria,German Restaurant,Café,Bakery,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
86,11.553128,"(48.1613555, 11.5531282)",6.0,German Restaurant,Trattoria/Osteria,Italian Restaurant,Bakery,Café,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner


In [190]:
munich_merged.loc[munich_merged['Cluster Labels'] == 7, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,11.633451,"(48.1234833, 11.6334511)",7.0,Gastropub,Café,Bakery,Fast Food Restaurant,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
20,11.634278,"(48.1266498, 11.6342782)",7.0,Asian Restaurant,Bakery,Italian Restaurant,Gastropub,Bavarian Restaurant,Café,Fast Food Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Diner
23,11.642630,"(48.1579231, 11.6426305)",7.0,Bakery,Greek Restaurant,Italian Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop
32,11.475596,"(48.125809, 11.4755958)",7.0,Bakery,Greek Restaurant,Gastropub,Doner Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Donut Shop
55,11.558598,"(48.1823734, 11.558598)",7.0,Bakery,Fast Food Restaurant,Greek Restaurant,Asian Restaurant,Doner Restaurant,Restaurant,Cafeteria,Falafel Restaurant,English Restaurant,Dim Sum Restaurant
56,11.575043,"(48.1823848, 11.5750432)",7.0,Bakery,Greek Restaurant,Burger Joint,German Restaurant,Gastropub,Thai Restaurant,Doner Restaurant,Xinjiang Restaurant,Deli / Bodega,Dim Sum Restaurant
57,11.516728,"(48.1810707, 11.5167276)",7.0,Bakery,Fast Food Restaurant,Burrito Place,German Restaurant,Trattoria/Osteria,Asian Restaurant,Pizza Place,Italian Restaurant,Food,Dim Sum Restaurant
58,11.507575,"(48.1807407, 11.5075745)",7.0,Bakery,Pizza Place,Asian Restaurant,Fast Food Restaurant,Italian Restaurant,Sandwich Place,Burrito Place,Food,Deli / Bodega,Gastropub
69,11.469372,"(48.16257, 11.4693724)",7.0,Bakery,Italian Restaurant,Xinjiang Restaurant,Falafel Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
72,11.605344,"(48.104872, 11.6053443)",7.0,Bakery,Fast Food Restaurant,Pizza Place,Asian Restaurant,Café,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [191]:
munich_merged.loc[munich_merged['Cluster Labels'] == 8, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,11.405779,"(48.1801726, 11.405779)",8.0,German Restaurant,Italian Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
95,11.683367,"(48.1415248, 11.683367)",8.0,Indian Restaurant,Comfort Food Restaurant,German Restaurant,Italian Restaurant,Xinjiang Restaurant,English Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant
102,11.518078,"(48.0765906, 11.5180779)",8.0,German Restaurant,Indian Restaurant,Deli / Bodega,Italian Restaurant,Xinjiang Restaurant,Ethiopian Restaurant,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop


In [192]:
munich_merged.loc[munich_merged['Cluster Labels'] == 9, munich_merged.columns[[1] + list(range(5, munich_merged.shape[1]))]]

,lng,latlng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
100,11.562029,"(48.1059355, 11.5620294)",9.0,Gastropub,Café,Xinjiang Restaurant,Ethiopian Restaurant,Deli / Bodega,Dim Sum Restaurant,Diner,Doner Restaurant,Donut Shop,Dumpling Restaurant
